In [ ]:
! pip3 install matplotlib numpy pandas pylzma ipykernel jupyter torch

  Using cached pylzma-0.5.0.tar.gz (4.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached jupyterlab-4.3.5-py3-none-any.whl.metadata (16 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-202

In [2]:
! pip3 install ipykernel jupyter torch

  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached torch-2.6.0-cp39-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached jupyterlab-4.3.5-py3-none-any.whl.metadata (16 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.me

In [3]:
! python3 -m ipykernel install --user --name=.venv --display-name="venv-gpt"

Installed kernelspec .venv in /Users/padraigobrien/Library/Jupyter/kernels/.venv


In [1]:
! jupyter notebook

/Users/padraigobrien/Downloads/API_LLM/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[I 2025-03-19 21:09:18.227 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2025-03-19 21:09:18.230 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2025-03-19 21:09:18.232 ServerApp] jupyterlab | extension was successfully linked.
[I 2025-03-19 21:09:18.234 ServerApp] notebook | extension was successfully linked.
[I 2025-03-19 21:09:18.411 ServerApp] notebook_shim | extension was successfully linked.
[I 2025-03-19 21:09:18.460 ServerApp] notebook_shim | extension was successfully loaded.
[I 2025-03-19 21:09:18.461 ServerApp] jupyter_lsp | extension was successfully loaded.
[I 2025-03-19 21:09:18.462 ServerApp] jupyter_server_terminals | extension was successfull

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'mps' if torch.mps.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

mps


In [82]:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [83]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [84]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x,y = x.to(device), y.to(device)

    return x,y

x,y = get_batch('train')
print('inputs: ')
print(x.shape)
print(x)
print('targets:')
print(y)

inputs: 
torch.Size([4, 8])
tensor([[66, 66, 58, 57, 62, 54, 73, 58],
        [58,  1, 43, 68, 71, 56, 58, 71],
        [ 1, 68, 67, 58,  0, 68, 59,  1],
        [47, 62, 79, 54, 71, 57, 24,  3]], device='mps:0')
targets:
tensor([[66, 58, 57, 62, 54, 73, 58, 65],
        [ 1, 43, 68, 71, 56, 58, 71, 58],
        [68, 67, 58,  0, 68, 59,  1, 73],
        [62, 79, 54, 71, 57, 24,  3,  0]], device='mps:0')


In [85]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(28)
when input is tensor([80, 28]) target is tensor(39)
when input is tensor([80, 28, 39]) target is tensor(42)
when input is tensor([80, 28, 39, 42]) target is tensor(39)
when input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


In [86]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [87]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embeddings_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indices in the current context

        for _ in range(max_new_tokens):
            # Get the predictions
            logits, loss = self.forward(index)
            # Focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1) # (B,C)
            # sample form the distribution
            index_next = torch.multinomial(probs, num_samples= 1) # (B,1)
            #append the sampled index to the running sequence
            index = torch.cat((index, index_next), dim = 1) # (B,T+1)
        return index
        

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
genererated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(genererated_chars)


V
yqQ;7xXs*AaHLYX,ftHEGcFH?yWfe;axESMnEo!_;N﻿:.qy'2nc!s9bjY!5!7DYfF!zbq." Wu"joQTvAc5HYxX;a5Qhoj!bC5[(RX7
tGk,Iaf;Yb40uFZt6;8*BG﻿v6M"EUWN[fZ.m_BObN]GOJQ&:x5sJeTA,cPv06VnT"1tyAt24Q.h[gcS2V!n?Ght 4wS,6 -yjZu:x Rj)3[sp5)micZQexUC)UWk&s)MxD,j[e4xtK5C300A"lC&Opa"zvkBm4!z_S,)GMglb6H8x5*AUH1TSp)0gVJDB ;I"!bXi!VlmOH&:qdg.jVJ"]'v;cUW﻿a
k6"P
﻿nL C)uFtb
y: hY0]4)?qj[DPuT,ab2seRKEc,mTeHLLJQs;_."EK;p)op3FSvA.vWgEtq4(0J_6qVvcxLJ-n.IK;.u'R?[)H8o?Xs7GNM
eK!,61MNTXP7P]cE9My'IEKzySMK9wD;af*97soN.0QTX ag'sUWXZ'DF[


In [93]:
#Create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    
    # Sample a batch of data
    xb, yb = get_batch('train')


    # evaluate the loss
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step 0, train loss: 4.0228, val loss: 4.0174
step 250, train loss: 3.9833, val loss: 3.9730
step 500, train loss: 3.9620, val loss: 3.9095
step 750, train loss: 3.9019, val loss: 3.8958
3.9680428504943848


In [89]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
genererated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(genererated_chars)


Te['DhvoZ!nw:fh*
t4W[;?w﻿i'bnC;TKA.jE_Ftmej5hi.7V-w:Woicf&RKEdUJ
-*8*LAVJQX5bnEKz:x)6H&"rIL﻿gT[fnQW4()9my)rNzm_C)f!zNMY
CNFVoDFn,c!4C L v39aH_﻿p*7IK;c)U a2n﻿CFjY3r: a[1pJlCUi6me﻿J5jjjdE,PxEKz]f9,cDTVn0Ou2nycRnA_M
t6OZl6hiWH,288B;4;a?﻿J"Ez]8zHy')y[Y06c ,-6O[g&nUIZJD8u]58jcCz
hAb,IENt 2
t;AV cN_r,PnLKvW061z;Iz&p"C!T2MNZ1L 7VoC;7exc2'1qAfFJubToN7,.?*UD;NMNN[&E6,Qv2g.T]Bbnjtv3dE
gq ,D2rvCQeFRdIoj8kmi:n?*BQHzEG,V6P'jGJD83DhywH*
IWi-?SOHjE
pFJUC k)ovMyg*UZtbC;_FJQs2nEK;JkE﻿C;73f,I8*WYMN4:V[?Dy;yt KTe4
